# 텐서플로우기반 케라스 진행

## 데이터

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터 공급
train_data_gen = ImageDataGenerator(rescale         = 1/255,   # 이미지 정규화
                                    horizontal_flip = True,    # 수평 대칭
                                    width_shift_range  = 0.1,  # 전체 크기의 10% 수준으로 좌우이동
                                    height_shift_range = 0.1,  # 전체 크기의 10% 수준으로 상하이동
                                    # 기타 부플리기 옶션 배제
                                    )
test_data_gen  = ImageDataGenerator(rescale         = 1/255)   # 이미지 정규화

In [3]:
# ImageDataGenerator를 이용 -> 공급자 구성
train_generator = train_data_gen.flow_from_directory(
    '/content/drive/MyDrive/데이터분석스쿨2기/share/12.데이터활용및과학방법론-12/res2/data-ch20/train'
    ,target_size = (150, 150) # 이미지 크기
    ,class_mode  = 'binary'   # 이진 분류
    ,batch_size  = 5          # 배치 사이즈
)
test_generator  = test_data_gen.flow_from_directory(
    '/content/drive/MyDrive/데이터분석스쿨2기/share/12.데이터활용및과학방법론-12/res2/data-ch20/test'
    ,target_size = (150, 150) # 이미지 크기
    ,class_mode  = 'binary'   # 이진 분류
    ,batch_size  = 5          # 배치 사이즈
)

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


## 인공신경망 구축 - 업스트림 다운및 조정

- 사전 학습된 업스트림모델 사용
    - 탐색후 사용
    - VGGNet 사용 (16)
        - 2014년 이미지 인식대회 준우승
        - 혁신적인 구조로 각광 받음
- 다운스트림 다운로드 => 출력층관련 신경망 추가(변환, 대체)등 수행 -> 모델 생성

In [4]:
# 1. 업스트림 모델 가져오기
from tensorflow.keras.applications import VGG16

In [5]:
# 2. 업스트림 모델 생성
'''
include_top = True,      :
    - True  : 업스트림의 신경망의 가장 마지막층(출력층)을 포함시킨다 -> 1000개로 분류
    - False : 포함 X

weights     = 'imagenet',
    - 제공되는 가중치를 그대로 사용(기본값)
    - 업스트립의 가중치증 특정한 값 사용

input_shape = None,
    - 업스트림이 학습할때 사용한 이미지가 아닌 다른 이미지를 사용할 경우-> 크기가 다른경우, 채널수가 다를 경우
    - 치매 CT 사진 => (150,150,3)
'''
pretrained_model = VGG16( include_top=False, weights='imagenet', input_shape=(150,150,3))

58889256/58889256 [==============================] - 1s 0us/step


In [6]:
# 3. 모델 요약
'''
    - 입력층의 shape 변경 확인
    - 파라미터수 체크
    - 현 파라미터는 모두 학습 가능한 것
    - 출력층 X
'''
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [7]:
# 4. 업스트림 모델 가중치 고정
# Non-trainable params: 14714688 (56.13 MB) <= 변경됨
pretrained_model.trainable = False # 가중치 유지, 업스트림 모델 훈련 X
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

## 인공신경망 구축 - 다운스트림 구현 - 치매 진단 모델

- 레이어 설계

```
업스트림   (None, 4, 4, 512)
flattern (None, 8192)     # 4*4*512
fc       (None, 64)
relu 활성화함수 Activation('relu')
dropout(0.5)
ouput    (None, 1)
시그모이드활성화함수 Activation('sigmoid')
```

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout

In [9]:
# 최종 모델
fine_tuning_model = Sequential()

fine_tuning_model.add( pretrained_model ) # 업스트림 모델 연결
# 실습, 커스텀 신경망 추가 6개 층을 추가, 실습 : 3분
fine_tuning_model.add( Flatten() ) # 4D->2D, (None, 8192)
fine_tuning_model.add( Dense(64) ) # (None, 8192) -> (None, 64)
fine_tuning_model.add( Activation('relu') ) # 활성화 함수 처리
fine_tuning_model.add( Dropout(0.5) ) # 과적합 방지 처리
fine_tuning_model.add( Dense(1) )  # (None, 64) -> (None, 1)
fine_tuning_model.add( Activation('sigmoid') ) # 이진분류 (0|1)

# Trainable params: 524417 (2.00 MB) <= 훈련 대상
fine_tuning_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 activation (Activation)     (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
 activation_1 (Activation)   (None, 1)                 0

## 컴파일

In [10]:
from tensorflow.keras import optimizers, metrics

fine_tuning_model.compile(
    optimizer= optimizers.Adam(learning_rate=0.0002), # 학습률 간격을 조밀하게 구성
    loss     = 'binary_crossentropy',
    metrics  = ['accuracy']
)

## 학습

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(patience=5)

In [12]:
import tensorflow as tf

In [13]:
with tf.device('/device:GPU:0'):
    history = fine_tuning_model.fit(train_generator, # x,y 공급 지원
                                    epochs    = 30,
                                    callbacks = [early_stopping],
                        validation_data = test_generator, # 검증용 데이터는 테스트용으로 임시 공급
                                    validation_steps = 10 # 검증에 사용하는 양
                                    )
# 24회차에서 학습 종료(조기학습종료)

Epoch 1/30
32/32 [==============================] - 48s 1s/step - loss: 0.6916 - accuracy: 0.6187 - val_loss: 0.7288 - val_accuracy: 0.4800
Epoch 2/30
32/32 [==============================] - 9s 293ms/step - loss: 0.6161 - accuracy: 0.7000 - val_loss: 0.5002 - val_accuracy: 0.8000
Epoch 3/30
32/32 [==============================] - 5s 146ms/step - loss: 0.4421 - accuracy: 0.8000 - val_loss: 0.3997 - val_accuracy: 0.7800
Epoch 4/30
32/32 [==============================] - 3s 108ms/step - loss: 0.4254 - accuracy: 0.8188 - val_loss: 0.4128 - val_accuracy: 0.8400
Epoch 5/30
32/32 [==============================] - 3s 100ms/step - loss: 0.3495 - accuracy: 0.8687 - val_loss: 0.2980 - val_accuracy: 0.9400
Epoch 6/30
32/32 [==============================] - 3s 89ms/step - loss: 0.3406 - accuracy: 0.8750 - val_loss: 0.3552 - val_accuracy: 0.8600
Epoch 7/30
32/32 [==============================] - 2s 70ms/step - loss: 0.3473 - accuracy: 0.8687 - val_loss: 0.3924 - val_accuracy: 0.8000
Epoch 8/30

In [15]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])